In [ ]:
# API_KEY = "9626ec2105ea082042fdda6db36f060dd228f2edc6229e8c5cd1cc3b2c673385"
API_KEY = "f9170d7223a9d2ba983443847320aa658edcc210742ee811ca09e1ab0dd956c9"

In [ ]:
ticker = 'AAPL'

## Libraries

In [ ]:
import requests
import json
import pandas as pd
from sec_api import QueryApi

### Query

In [ ]:
query_api = QueryApi(api_key=API_KEY)

# fetch all 10-Q and 10-K
query = { 
    "query": {
        "query_string": {
            "query": "(formType:\"10-Q\" OR formType:\"10-K\") AND ticker:"+ticker
        }
    },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" }}]
}

query_result = query_api.get_filings(query)

### get the accessioin number

In [ ]:
accession_numbers = []

# extract accession numbers of each filing
for filing in query_result['filings']:
    accession_numbers.append(filing['accessionNo'])


### get the XBLR files

In [ ]:
# get XBRL-JSON for a given accession number
def get_xbrl_json(accession_no):
    '''
    Helper function to get XBRL-JSON version of a filing by providing its accession number
    '''
    # XBRL-to-JSON converter API endpoint
    xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

    request_url = xbrl_converter_api_endpoint + "?accession-no=" + accession_no + "&token=" + API_KEY
    
    response_tmp = requests.get(request_url)
    
    return json.loads(response_tmp.text)

### Merge two statement

In [ ]:
# merge two statements into one statement
# row indices of both statements have to be the same
# statement_b represents the most recent statement.
def merge_statements(statement_a, statement_b):
    return statement_a.merge(statement_b,
                             how="outer",
                             left_index=True,
                             right_index=True,
                             suffixes=('_left', ''))

### Clean data

In [ ]:
# clean statement
# drop duplicate columns (= column name ends with "_left"), drop key_0 column, drop columns with +5 NaNs
def clean_data_statement(statement):
    for column in statement:
        
        #column has more than 5 NaN values
        is_nan_column = statement[column].isna().sum() > 5
        
        if column.endswith('_left') or column == 'key_0' or is_nan_column:
            statement = statement.drop(column, axis=1)
        
    # rearrange columns so that first column represents first quarter
    # e.g. 2018, 2019, 2020 - and not 2020, 2019, 2018
    sorted_columns = sorted(statement.columns.values)
    
    return statement[sorted_columns]

### Income Statement

In [ ]:
# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}
    
    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []
        
        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)
        
        income_statement_store[usGaapItem] = pd.Series(values, index=indicies)
        
    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a data range
    return income_statement .T

### Balance Sheet

In [ ]:
# convert XBRL-JSON of balance sheet to pandas dataframe
def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}
    
    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []
        
        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['instant']
                
                # avoid duplicate indices with same values
                if index in indicies:
                    continue
                
                # add 0 if value is nil
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])
                    
                indicies.append(index)
            
            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies)
            
    balance_sheet = pd.DataFrame(balance_sheet_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date instant
    return balance_sheet.T


### Cash Flow Statement

In [ ]:
# generate the cash flow statement
def get_cash_flow_statement(xbrl_json):
    cash_flows_store = {}
    
    for usGaapItem in xbrl_json['StatementsOfCashFlows']:
        values = []
        indicies = []
        
        for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:
            # only consider items without segment
            if 'segment' not in fact:
                #check if data instant or date range is present
                if 'instant' in fact['period']:
                    index = fact['period']['instant']
                else:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']
            
                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])
                
                indicies.append(index)
        
        cash_flows_store[usGaapItem] = pd.Series(values, index=indicies)
    
    cash_flows = pd.DataFrame(cash_flows_store)
    return cash_flows.T


### Code

In [ ]:
previous_income_statement_set = False
income_statement_final = None

previous_balance_sheet_set = False
balance_sheet_final = None

previous_cashflow_statement_set = False
cashflow_statement_final = None

for accession_no in accession_numbers[0:9]: # doesn't work with filings filed before 2017

    # get XBRL-JSON of 10-Q or 10-K filing by accession number
    xbrl_json_data = get_xbrl_json(accession_no)
    
    print(xbrl_json_data)
    
    # convert XBRL-JSON to a pandas dataframe
    income_statement_uncleaned = get_income_statement(xbrl_json_data)
    balance_sheet_uncleaned = get_balance_sheet(xbrl_json_data)
    cashflow_statement_uncleaned = get_cash_flow_statement(xbrl_json_data)
    
    # clean the data
    income_statement_cleaned = clean_data_statement(income_statement_uncleaned)
    balance_sheet_cleaned = clean_data_statement(balance_sheet_uncleaned)
    cashflow_statement_cleaned = clean_data_statement(cashflow_statement_uncleaned)
    
    # merge new income statement with previously generated income statement
    if previous_income_statement_set and previous_balance_sheet_set and previous_cashflow_statement_set:
        income_statement_final = clean_data_statement(merge_statements(income_statement_final, income_statement_cleaned))
        balance_sheet_final = clean_data_statement(merge_statements(balance_sheet_final, balance_sheet_cleaned))
        cashflow_statement_final = clean_data_statement(merge_statements(cashflow_statement_final, cashflow_statement_cleaned))
    else:
        income_statement_final = income_statement_cleaned
        balance_sheet_final = balance_sheet_cleaned
        cashflow_statement_final = cashflow_statement_cleaned
        previous_income_statement_set = True
        previous_balance_sheet_set = True
        previous_cashflow_statement_set = True

### New Code

In [ ]:
previous_income_statement_set = False
income_statement_final = None

previous_balance_sheet_set = False
balance_sheet_final = None

previous_cashflow_statement_set = False
cashflow_statement_final = None

for accession_no in accession_numbers[0:9]: # doesn't work with filings filed before 2017

    # get XBRL-JSON of 10-Q or 10-K filing by accession number
    xbrl_json_data = get_xbrl_json(accession_no)
    
    # convert XBRL-JSON to a pandas dataframe
    income_statement_uncleaned = get_income_statement(xbrl_json_data)
    balance_sheet_uncleaned = get_balance_sheet(xbrl_json_data)
    cashflow_statement_uncleaned = get_cash_flow_statement(xbrl_json_data)
    
    # clean the data
#     income_statement_cleaned = clean_data_statement(income_statement_uncleaned)
#     balance_sheet_cleaned = clean_data_statement(balance_sheet_uncleaned)
#     cashflow_statement_cleaned = clean_data_statement(cashflow_statement_uncleaned)
    
    # merge new income statement with previously generated income statement
    if previous_income_statement_set and previous_balance_sheet_set and previous_cashflow_statement_set:
        income_statement_final = merge_statements(income_statement_final, income_statement_uncleaned)
        balance_sheet_final = merge_statements(balance_sheet_final, balance_sheet_uncleaned)
        cashflow_statement_final = merge_statements(cashflow_statement_final, cashflow_statement_uncleaned)
    else:
        income_statement_final = income_statement_uncleaned
        balance_sheet_final = balance_sheet_uncleaned
        cashflow_statement_final = cashflow_statement_uncleaned
        previous_income_statement_set = True
        previous_balance_sheet_set = True
        previous_cashflow_statement_set = True

In [ ]:
income_statement_final.to_csv(f"{ticker}_income_statement.csv")
balance_sheet_final.to_csv(f"{ticker}_balance_sheet.csv")
cashflow_statement_final.to_csv(f"{ticker}_cashflow_statement.csv")

In [1]:
import requests
import json
import pandas as pd
from sec_api import QueryApi
from pymongo import MongoClient

In [5]:
class SEC_API:
    def __init__(self, ticker):
        self.API_KEY = "f9170d7223a9d2ba983443847320aa658edcc210742ee811ca09e1ab0dd956c9"
        
        query_api = QueryApi(api_key=self.API_KEY)

        # fetch all 10-Q and 10-K
        query = { 
            "query": {
                "query_string": {
                    "query": "(formType:\"10-Q\" OR formType:\"10-K\") AND ticker:"+ticker
                }
            },
            "from": "0",
            "size": "20",
            "sort": [{ "filedAt": { "order": "desc" }}]
        }

        query_result = query_api.get_filings(query)
    
        self.accession_numbers = []
        # extract accession numbers of each filing
        for filing in query_result['filings']:
            self.accession_numbers.append(filing['accessionNo'])
    
    # get XBRL-JSON for a given accession number
    def get_xbrl_json(self, accession_no):
        '''
        Helper function to get XBRL-JSON version of a filing by providing its accession number
        '''
        # XBRL-to-JSON converter API endpoint
        xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

        request_url = xbrl_converter_api_endpoint + "?accession-no=" + accession_no + "&token=" + API_KEY

        response_tmp = requests.get(request_url)

        return json.loads(response_tmp.text)
    
    # merge two statements into one statement
    # row indices of both statements have to be the same
    # statement_b represents the most recent statement.
    def merge_statements(self, statement_a, statement_b):
        return statement_a.merge(statement_b,
                                 how="outer",
                                 left_index=True,
                                 right_index=True,
                                 suffixes=('_left', ''))
    
    # clean statement
    # drop duplicate columns (= column name ends with "_left"), drop key_0 column, drop columns with +5 NaNs
    def clean_data_statement(self, statement):
        for column in statement:

            #column has more than 5 NaN values
            is_nan_column = statement[column].isna().sum() > 5

            if column.endswith('_left') or column == 'key_0' or is_nan_column:
                statement = statement.drop(column, axis=1)

        # rearrange columns so that first column represents first quarter
        # e.g. 2018, 2019, 2020 - and not 2020, 2019, 2018
        sorted_columns = sorted(statement.columns.values)

        return statement[sorted_columns]
    
    # convert XBRL-JSON of income statement to pandas dataframe
    def get_income_statement(self, xbrl_json):
        income_statement_store = {}

        # iterate over each US GAAP item in the income statement
        for usGaapItem in xbrl_json['StatementsOfIncome']:
            values = []
            indicies = []

            for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
                # only consider items without segment.
                if 'segment' not in fact:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                    # ensure no index duplicates are created
                    if index not in indicies:
                        values.append(fact['value'])
                        indicies.append(index)

            income_statement_store[usGaapItem] = pd.Series(values, index=indicies)

        income_statement = pd.DataFrame(income_statement_store)
        # switch columns and rows so that US GAAP items are rows and each column header represents a data range
        return income_statement.T
    
    # convert XBRL-JSON of balance sheet to pandas dataframe
    def get_balance_sheet(self, xbrl_json):
        balance_sheet_store = {}

        for usGaapItem in xbrl_json['BalanceSheets']:
            values = []
            indicies = []

            for fact in xbrl_json['BalanceSheets'][usGaapItem]:
                # only consider items without segment.
                if 'segment' not in fact:
                    index = fact['period']['instant']

                    # avoid duplicate indices with same values
                    if index in indicies:
                        continue

                    # add 0 if value is nil
                    if "value" not in fact:
                        values.append(0)
                    else:
                        values.append(fact['value'])

                    indicies.append(index)

                balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies)

        balance_sheet = pd.DataFrame(balance_sheet_store)
        # switch columns and rows so that US GAAP items are rows and each column header represents a date instant
        return balance_sheet.T
    
    # generate the cash flow statement
    def get_cash_flow_statement(self, xbrl_json):
        cash_flows_store = {}

        for usGaapItem in xbrl_json['StatementsOfCashFlows']:
            values = []
            indicies = []

            for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:
                # only consider items without segment
                if 'segment' not in fact:
                    #check if data instant or date range is present
                    if 'instant' in fact['period']:
                        index = fact['period']['instant']
                    else:
                        index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                    # avoid duplicate indicies with same values
                    if index in indicies:
                        continue

                    if "value" not in fact:
                        values.append(0)
                    else:
                        values.append(fact['value'])

                    indicies.append(index)

            cash_flows_store[usGaapItem] = pd.Series(values, index=indicies)

        cash_flows = pd.DataFrame(cash_flows_store)
        return cash_flows.T
    
    
    def run(self):
        previous_income_statement_set = False
        self.income_statement_final = None

        previous_balance_sheet_set = False
        self.balance_sheet_final = None

        previous_cashflow_statement_set = False
        self.cashflow_statement_final = None

        for accession_no in self.accession_numbers[0:9]: # doesn't work with filings filed before 2017

            # get XBRL-JSON of 10-Q or 10-K filing by accession number
            xbrl_json_data = self.get_xbrl_json(accession_no)

            # convert XBRL-JSON to a pandas dataframe
            income_statement_uncleaned = self.get_income_statement(xbrl_json_data)
            balance_sheet_uncleaned = self.get_balance_sheet(xbrl_json_data)
            cashflow_statement_uncleaned = self.get_cash_flow_statement(xbrl_json_data)

            # clean the data
            income_statement_cleaned = self.clean_data_statement(income_statement_uncleaned)
            balance_sheet_cleaned = self.clean_data_statement(balance_sheet_uncleaned)
            cashflow_statement_cleaned = self.clean_data_statement(cashflow_statement_uncleaned)

            # merge new income statement with previously generated income statement
            if previous_income_statement_set and previous_balance_sheet_set and previous_cashflow_statement_set:
                self.income_statement_final = self.clean_data_statement(self.merge_statements(self.income_statement_final, income_statement_cleaned))
                self.balance_sheet_final = self.clean_data_statement(self.merge_statements(self.balance_sheet_final, balance_sheet_cleaned))
                self.cashflow_statement_final = self.clean_data_statement(self.merge_statements(self.cashflow_statement_final, cashflow_statement_cleaned))
            else:
                self.income_statement_final = income_statement_cleaned
                self.balance_sheet_final = balance_sheet_cleaned
                self.cashflow_statement_final = cashflow_statement_cleaned
                previous_income_statement_set = True
                previous_balance_sheet_set = True
                previous_cashflow_statement_set = True
        
    
    def save_csv(self):
        self.income_statement_final.to_csv(f"{ticker}_income_statement.csv")
        self.balance_sheet_final.to_csv(f"{ticker}_balance_sheet.csv")
        self.cashflow_statement_final.to_csv(f"{ticker}_cashflow_statement.csv")

In [6]:
sec = SEC_API(ticker="A")

In [7]:
sec.run()

NameError: name 'API_KEY' is not defined